In [1]:
import pandas as pd
pd.set_option('display.max_rows', None) # Print all rows

In [2]:
# FIX SSL: CERTIFICATE_VERIFY_FAILED with Python3
# https://stackoverflow.com/a/49174340
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
url = 'https://www.juntadeandalucia.es/institutodeestadisticaycartografia/badea/stpivot/stpivot/Print?cube=013d884c-cfc5-4c21-9e78-e26225682afd&type=3&foto=si&ejecutaDesde=&codConsulta=38668&consTipoVisua=JP'
fields = ['Lugar de residencia', 'Medida', 'Valor']
df = pd.read_csv(url, delimiter=';', usecols=fields, decimal=',')

In [4]:
df[df['Lugar de residencia'] == 'Huelva']

,Lugar de residencia,Medida,Valor
0,Huelva,Población,513170
1,Huelva,Confirmados PDIA,43654
2,Huelva,Confirmados PDIA 14 días,1972
3,Huelva,Tasa PDIA 14 días,"384,2781144649921"
4,Huelva,Confirmados PDIA 7 días,888
5,Huelva,Tasa PDIA 7 dias,"173,0420718280492"
6,Huelva,Total Confirmados,43853
7,Huelva,Curados,38144
8,Huelva,Fallecidos,402


In [5]:
df['Valor'] = pd.to_numeric(df['Valor'],errors='coerce')

In [6]:
dfAnterior = pd.read_csv('covid_huelva.csv', index_col='Lugar de residencia')

In [7]:
# Confirmados PCR provincia Huelva
p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en la provincia de Huelva: ', p)

Confirmados PCR en la provincia de Huelva:  43654.0


/tmp/ipykernel_3724/1396922249.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [8]:
# Confirmados PCR Huelva capitalpd
c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en Huelva capital: ', c)

Confirmados PCR en Huelva capital:  12158.0


/tmp/ipykernel_3724/2509482988.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [9]:
c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]

/tmp/ipykernel_3724/2690025379.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
/tmp/ipykernel_3724/2690025379.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
/tmp/ipykernel_3724/2690025379.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]


In [10]:
print('Huelva (capital)\n')
#print('Confirmados PCR 7 días x 1,000 habitantes: ', f"{c7/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c7:.0f}", 'personas en los últimos 7 días', '\n')
#print('Confirmados PCR 14 días x 1,000 habitantes: ', f"{c14/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c14:.0f}", 'personas en los últimos 14 días')

Huelva (capital)

Un positivo PCR cada 639 personas en los últimos 7 días 

Un positivo PCR cada 284 personas en los últimos 14 días


In [11]:
filtro = ((df['Medida'] == 'Confirmados PDIA')| (df['Medida'] == 'Confirmados PDIA 7 días') | (df['Medida'] == 'Población' ) | (df['Medida'] == 'Confirmados PDIA 14 días')) & (df['Valor'] > 0)
tabla = pd.pivot_table(df.loc[filtro], values='Valor', index='Lugar de residencia', columns='Medida').dropna()

In [12]:
# Reorder columns
tabla = tabla[['Confirmados PDIA', 'Confirmados PDIA 7 días', 'Confirmados PDIA 14 días', 'Población']]

In [13]:
tabla['tasa7'] = tabla['Confirmados PDIA 7 días']/tabla['Población']*100000
tabla['tasa14'] = tabla['Confirmados PDIA 14 días']/tabla['Población']*100000

In [14]:
tabla['Diferencia PDIA'] = tabla['Confirmados PDIA'] - dfAnterior['Confirmados PDIA']

In [15]:
# Nuevos casos desde la última captura
tabla[tabla['Diferencia PDIA'] != 0].sort_values('Diferencia PDIA', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Huelva,43654.0,888.0,1972.0,513170.0,173.042072,384.278114,202.0
Huelva-Costa,25760.0,580.0,1253.0,289548.0,200.312211,432.743448,124.0
Condado-Campiña,13628.0,220.0,545.0,156231.0,140.817123,348.842419,59.0
Huelva (capital),12158.0,225.0,507.0,143837.0,156.427067,352.482324,38.0
Lepe,2903.0,85.0,155.0,27880.0,304.878049,555.954089,22.0
Isla Cristina,2979.0,80.0,167.0,21393.0,373.954097,780.629178,22.0
Moguer,1881.0,30.0,66.0,21867.0,137.193031,301.824667,17.0
Sierra de Huelva-Andévalo Central,3859.0,72.0,153.0,67391.0,106.839192,227.033283,16.0
Ayamonte,1949.0,62.0,154.0,21104.0,293.783169,729.719484,13.0


In [16]:
if tabla[tabla['Diferencia PDIA'] != 0]['Diferencia PDIA'].sum() != 0:
  # Save to csv
  tabla.to_csv('covid_huelva.csv', encoding='utf-8', header=True, index=True)  

In [17]:
tabla.sort_values('tasa14', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Cañaveral de León,26.0,5.0,11.0,396.0,1262.626263,2777.777778,0.0
Linares de la Sierra,10.0,5.0,6.0,262.0,1908.396947,2290.076336,1.0
Alájar,34.0,7.0,15.0,759.0,922.266140,1976.284585,3.0
Almendro (El),48.0,5.0,9.0,840.0,595.238095,1071.428571,5.0
Rosal de la Frontera,60.0,13.0,14.0,1697.0,766.057749,824.985268,2.0
Isla Cristina,2979.0,80.0,167.0,21393.0,373.954097,780.629178,22.0
Ayamonte,1949.0,62.0,154.0,21104.0,293.783169,729.719484,13.0
Escacena del Campo,218.0,10.0,15.0,2287.0,437.254045,655.881067,4.0
Cartaya,1788.0,60.0,131.0,20083.0,298.760145,652.292984,10.0


In [18]:
# Peso de la tasa de la última semana en relación con la tasa a 14 días 
# Mejor cuanto menor. Valor 1 significa que no había casos antes de 7 días.
tabla['7/14'] = tabla['tasa7'] / tabla['tasa14']
tabla.fillna(0).sort_values('7/14', ascending=True)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA,7/14
Lugar de residencia,,,,,,,,
Paterna del Campo,191.0,1.0,14.0,3457.0,28.926815,404.975412,0.0,0.071429
Cortegana,284.0,1.0,14.0,4602.0,21.729683,304.215558,0.0,0.071429
Cumbres Mayores,129.0,2.0,9.0,1749.0,114.351058,514.579760,0.0,0.222222
Trigueros,530.0,10.0,42.0,7862.0,127.194098,534.215212,3.0,0.238095
Bonares,444.0,6.0,25.0,6060.0,99.009901,412.541254,2.0,0.240000
Manzanilla,136.0,1.0,4.0,2118.0,47.214353,188.857413,0.0,0.250000
San Bartolomé de la Torre,354.0,4.0,15.0,3761.0,106.354693,398.830098,0.0,0.266667
Gibraleón,900.0,9.0,32.0,12737.0,70.660281,251.236555,2.0,0.281250
Bollullos Par del Condado,1253.0,17.0,60.0,14387.0,118.162230,417.043164,1.0,0.283333
